## Comparação CAPM vs Fama-French em Setores da B3

Disciplina: Finanças II  
Objetivo: Comparar o poder explicativo do CAPM (1 fator) e do Modelo Fama-French (3 fatores) nos retornos de três setores representativos da Bolsa Brasileira (B3), com características distintas quanto à sensibilidade econômica e volatilidade.

### Setores Analisados
| Setor                 | Características                                               | Tickers                  |
|-----------------------|---------------------------------------------------------------|--------------------------|
| **Varejo (Cíclico)**  | Alta sensibilidade ao ciclo econômico, forte variação com renda e consumo | MGLU3, LREN3, BHIA3 |
| **Bancário (Defensivo)** | Menor volatilidade, fluxos previsíveis, lucros resilientes | ITSA4, BBAS3, BBDC4 |
| **Elétrico (Estável)**   | Setor regulado, receitas previsíveis, baixo risco sistemático | ENBR3, CMIG4, TAEE11 |


### Metodologia

 1. Coleta de Dados: Preços históricos via `yfinance` (5 anos, dados diários)
2. Construção de Carteiras: Média simples dos retornos setoriais
3. Modelos de Regressão:
    - CAPM: $R_i - R_f = α + β(R_m - R_f) + ε$
    - Fama-French: $R_i - R_f = α + β_Mkt(R_m - R_f) + β_SMB(SMB) + β_HML(HML) + ε$
4. Análise Comparativa: Avaliação de R², Alphas, Betas e significância estatística

Link para o arquivo "nefin_factos.csv": https://nefin.com.br/data/risk_factors.html



In [39]:
import yfinance as yf
import pandas as pd
import numpy as np
import statsmodels.api as sm
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

Definindo os tickers para cada setor e período para análise

In [40]:
tickers_varejo = ['MGLU3.SA', 'LREN3.SA', 'BHIA3.SA']
tickers_bancos = ['ITSA4.SA', 'BBAS3.SA', 'BBDC4.SA']
tickers_eletrico = ['ELET3.SA', 'TAEE11.SA', 'CMIG4.SA']
ticker_mercado = '^BVSP'

In [41]:
start_date = datetime(2020, 6, 1) 
end_date = datetime(2025, 6, 30)

Baixando os dados, calculando retornos e limpando parte da base de dados

In [42]:
todos_tickers = tickers_varejo + tickers_bancos + tickers_eletrico + [ticker_mercado]

dados = yf.download(todos_tickers, start=start_date, end=end_date, 
                   interval='1d', progress=False)

# Debug: verificar estrutura dos dados
print("\nEstrutura dos dados retornados:")
print(f"Tipo: {type(dados)}")
print(f"Shape: {dados.shape}")
print(f"Colunas: {dados.columns.tolist()[:10]}")  # Primeiras 10 colunas
print(f"É MultiIndex? {isinstance(dados.columns, pd.MultiIndex)}")



Estrutura dos dados retornados:
Tipo: <class 'pandas.core.frame.DataFrame'>
Shape: (1263, 50)
Colunas: [('Close', 'BBAS3.SA'), ('Close', 'BBDC4.SA'), ('Close', 'BHIA3.SA'), ('Close', 'CMIG4.SA'), ('Close', 'ELET3.SA'), ('Close', 'ITSA4.SA'), ('Close', 'LREN3.SA'), ('Close', 'MGLU3.SA'), ('Close', 'TAEE11.SA'), ('Close', '^BVSP')]
É MultiIndex? True


In [43]:
precos = dados['Close']
retornos = precos.pct_change().dropna()


Separando por grupos

In [44]:
retornos_varejo = retornos[[t for t in tickers_varejo]]
retornos_bancos = retornos[[t for t in tickers_bancos]]
retornos_eletrico = retornos[[t for t in tickers_eletrico]]
retornos_mercado = retornos[ticker_mercado]

In [45]:
print('Retornos Varejo:', retornos_varejo.head())

Retornos Varejo: Ticker      MGLU3.SA  LREN3.SA  BHIA3.SA
Date                                    
2020-06-02   -0.0298    0.0278   -0.0112
2020-06-03    0.0094    0.0487    0.0151
2020-06-04   -0.0303    0.0086    0.0349
2020-06-05   -0.0172    0.0319    0.0645
2020-06-08    0.0493    0.0050    0.0438


Construção das carteiras por setor

A carteria será construída com uma média simples dos retornos: 

$$ R_{carteira, t} = \frac{1}{N}\sum_{i=1}^{N}R_{i,t}$$

In [46]:
carteira_varejo = retornos_varejo.mean(axis=1)
carteira_bancos = retornos_bancos.mean(axis=1)
carteira_eletrico = retornos_eletrico.mean(axis=1)

retorno_mercado = retornos_mercado.squeeze()

In [47]:
datas_comuns = carteira_varejo.index.intersection(carteira_bancos.index).intersection(carteira_eletrico.index).intersection(retorno_mercado.index)
carteira_varejo = carteira_varejo.loc[datas_comuns]
carteira_bancos = carteira_bancos.loc[datas_comuns]
carteira_eletrico = carteira_eletrico.loc[datas_comuns]
retorno_mercado = retorno_mercado.loc[datas_comuns]

print(f"Carteiras criadas com {len(datas_comuns)} observações alinhadas\n")

Carteiras criadas com 1262 observações alinhadas



In [48]:
stats = pd.DataFrame({
    'Varejo': carteira_varejo.describe(),
    'Bancos': carteira_bancos.describe(),
    'Elétrico': carteira_eletrico.describe(),
    'Mercado': retorno_mercado.describe()
})

print('Estatísticas Descritivas:')
display(stats.T)

Estatísticas Descritivas:


,count,mean,std,min,25%,50%,75%,max
Varejo,1262.0000,-0.0012,0.0323,-0.1183,-0.0217,-0.0030,0.0158,0.1335
Bancos,1262.0000,0.0006,0.0156,-0.0806,-0.0082,0.0005,0.0095,0.0762
Elétrico,1262.0000,0.0008,0.0136,-0.0503,-0.0073,0.0009,0.0092,0.0671
Mercado,1262.0000,0.0004,0.0118,-0.0512,-0.0067,0.0003,0.0077,0.0554


## Carregamento dos Fatores Fama-French

Dados retirados do Nefin

Fatores necessários:
- **Rm - Rf**: Prêmio de risco do mercado
- **SMB**: Prêmio de tamanho (empresas pequenas vs grandes)
- **HML**: Prêmio de valor (empresas value vs growth)
- **Risk_free**: Taxa livre de risco

In [49]:
fatores = pd.read_csv('nefin_factors.csv')

fatores['Date'] = pd.to_datetime(fatores['Date'])

fatores = fatores.set_index('Date')

fatores = fatores[['Rm_minus_Rf', 'SMB', 'HML', 'Risk_Free']]
fatores.columns = ['Rm_minus_Rf', 'SMB', 'HML', 'Risk_free']

fatores = fatores.loc[start_date:end_date]

In [50]:
display(fatores.head())

,Rm_minus_Rf,SMB,HML,Risk_free
Date,,,,
2020-06-01,0.0133,0.0277,0.0080,0.0001
2020-06-02,0.0261,0.0015,0.0160,0.0001
2020-06-03,0.0202,0.0081,-0.0001,0.0001
2020-06-04,0.0073,0.0295,0.0182,0.0001
2020-06-05,0.0107,-0.0011,-0.0059,0.0001


Cálculo dos Retornos Excedentes

Retorno Excedente = Retorno da Carteira - Taxa Livre de Risco

$$R_i - R_f$$

In [51]:
print("Fatores:", fatores.index[:5])
print("Carteira Varejo:", carteira_varejo.index[:5])
print("Carteira Bancos:", carteira_bancos.index[:5])

Fatores: DatetimeIndex(['2020-06-01', '2020-06-02', '2020-06-03', '2020-06-04',
               '2020-06-05'],
              dtype='datetime64[ns]', name='Date', freq=None)
Carteira Varejo: DatetimeIndex(['2020-06-02', '2020-06-03', '2020-06-04', '2020-06-05',
               '2020-06-08'],
              dtype='datetime64[ns]', name='Date', freq=None)
Carteira Bancos: DatetimeIndex(['2020-06-02', '2020-06-03', '2020-06-04', '2020-06-05',
               '2020-06-08'],
              dtype='datetime64[ns]', name='Date', freq=None)


In [52]:
# Alinhamento das datas

datas_comuns = carteira_varejo.index.intersection(fatores.index)
carteira_varejo = carteira_varejo.loc[datas_comuns]
carteira_bancos = carteira_bancos.loc[datas_comuns]
carteira_eletrico = carteira_eletrico.loc[datas_comuns]

fatores = fatores.loc[datas_comuns]

In [53]:
ret_exc_varejo = carteira_varejo - fatores['Risk_free']
ret_exc_bancos = carteira_bancos - fatores['Risk_free']
ret_exc_eletrico = carteira_eletrico - fatores['Risk_free']

comparacao = pd.DataFrame({
    'Ret. Varejo': carteira_varejo,
    'Ret. Exc. Varejo': ret_exc_varejo,
    'Ret. Bancos': carteira_bancos,
    'Ret. Exc. Bancos': ret_exc_bancos,
    'Ret. Elétrico': carteira_eletrico,
    'Ret. Exc. Elétrico': ret_exc_eletrico
}).head()

print('Retornos Brutos vs Excedentes:')
display(comparacao)

Retornos Brutos vs Excedentes:


,Ret. Varejo,Ret. Exc. Varejo,Ret. Bancos,Ret. Exc. Bancos,Ret. Elétrico,Ret. Exc. Elétrico
Date,,,,,,
2020-06-02,-0.0044,-0.0045,0.0429,0.0428,0.0126,0.0125
2020-06-03,0.0244,0.0243,0.0439,0.0438,-0.0021,-0.0022
2020-06-04,0.0044,0.0043,0.0103,0.0102,-0.0102,-0.0103
2020-06-05,0.0264,0.0263,0.0126,0.0125,0.0178,0.0177
2020-06-08,0.0327,0.0326,0.0407,0.0406,0.0415,0.0414


## Definição dos Modelos

CAPM (Capital Asset Pricing Model)

$$R_i - R_f = \alpha + \beta(R_m - R_f) + \varepsilon$$

**Alpha**: Retorno anormal (desempenho acima/abaixo do esperado)
**Beta**: Sensibilidade ao mercado (risco sistemático)
**R²**: Poder explicativo do modelo

### Fama-French

$$R_i - R_f = \alpha + \beta_{Mkt}(R_m - R_f) + \beta_{SMB}(SMB) + \beta_{HML}(HML) + \varepsilon$$

Fatores Adicionais:

- **SMB**: Prêmio de tamanho (Small Minus Big)
- **HML**: Prêmio de valor (High Minus Low)

In [54]:
def executar_capm(ret_exc_carteira, rm_minus_rf):
    X = sm.add_constant(rm_minus_rf)
    modelo = sm.OLS(ret_exc_carteira, X).fit()
    return modelo

def executar_ff(ret_exc_carteira, rm_minus_rf, smb, hml):
    X = pd.DataFrame({
        'Rm_minus_Rf': rm_minus_rf,
        'SMB': smb,
        'HML': hml
    })
    X = sm.add_constant(X)
    modelo = sm.OLS(ret_exc_carteira, X).fit()
    return modelo

## Execuções das Regressões

In [55]:
modelo_capm_varejo = executar_capm(ret_exc_varejo, fatores['Rm_minus_Rf'])
modelo_ff_varejo = executar_ff(ret_exc_varejo, fatores['Rm_minus_Rf'],
                               fatores['SMB'], fatores['HML'])

modelo_capm_bancos = executar_capm(ret_exc_bancos, fatores['Rm_minus_Rf'])
modelo_ff_bancos = executar_ff(ret_exc_bancos, fatores['Rm_minus_Rf'],
                               fatores['SMB'], fatores['HML'])

modelo_capm_eletrico = executar_capm(ret_exc_eletrico, fatores['Rm_minus_Rf'])
modelo_ff_eletrico = executar_ff(ret_exc_eletrico, fatores['Rm_minus_Rf'],
                                  fatores['SMB'], fatores['HML'])

print('Regressões concluídas')

Regressões concluídas


## Resultados detalhados - Setor Varejo

In [56]:
print('-' * 40)
print('Resultados - Setor Varejista')
print('-' * 40)

print('\nModelo CAPM:')
print(modelo_capm_varejo.summary())

print('\nModelo Fama-French:')
print(modelo_ff_varejo.summary())

----------------------------------------
Resultados - Setor Varejista
----------------------------------------

Modelo CAPM:
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.300
Model:                            OLS   Adj. R-squared:                  0.300
Method:                 Least Squares   F-statistic:                     540.7
Date:                Tue, 11 Nov 2025   Prob (F-statistic):          8.17e-100
Time:                        16:51:19   Log-Likelihood:                 2765.4
No. Observations:                1262   AIC:                            -5527.
Df Residuals:                    1260   BIC:                            -5517.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-----

## Resultados detalhados - Setor Bancário

In [57]:
print('-' * 40)
print('Resultados - Setor Bancário')
print('-' * 40)

print('\nModelo CAPM:')
print(modelo_capm_bancos.summary())

print('\nModelo Fama-French:')
print(modelo_ff_bancos.summary())

----------------------------------------
Resultados - Setor Bancário
----------------------------------------

Modelo CAPM:
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.518
Model:                            OLS   Adj. R-squared:                  0.518
Method:                 Least Squares   F-statistic:                     1356.
Date:                Tue, 11 Nov 2025   Prob (F-statistic):          4.61e-202
Time:                        16:51:19   Log-Likelihood:                 3921.9
No. Observations:                1262   AIC:                            -7840.
Df Residuals:                    1260   BIC:                            -7830.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
------

## Resultados Detalhados - Setor Elétrico

In [58]:
print('-' * 40)
print('Resultados - Setor Elétrico')
print('-' * 40)

print('\nModelo CAPM:')
print(modelo_capm_eletrico.summary())

print('\nModelo Fama-French:')
print(modelo_ff_eletrico.summary())

----------------------------------------
Resultados - Setor Elétrico
----------------------------------------

Modelo CAPM:
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.411
Model:                            OLS   Adj. R-squared:                  0.410
Method:                 Least Squares   F-statistic:                     877.9
Date:                Tue, 11 Nov 2025   Prob (F-statistic):          7.64e-147
Time:                        16:51:19   Log-Likelihood:                 3965.2
No. Observations:                1262   AIC:                            -7926.
Df Residuals:                    1260   BIC:                            -7916.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
------

## Resumo Comparativo

In [59]:
def extrair_resultados(modelo, nome_modelo):
    params = modelo.params
    pvalues = modelo.pvalues

    resultados = {
        'Modelo': nome_modelo,
        'Alpha': params['const'],
        'Alpha (p-valor)': pvalues['const'],
        'Beta_Mercado': params.get('Rm_minus_Rf', params.iloc[1]),
        'Beta_Mercado (p-valor)': pvalues.get('Rm_minus_Rf', pvalues.iloc[1]),
        'R²': modelo.rsquared,
        'R² Ajustado': modelo.rsquared_adj
    }

    if 'SMB' in params:
        resultados['Beta_SMB'] = params['SMB']
        resultados['Beta_SMB (p-valor)'] = pvalues['SMB']
    else:
        resultados['Beta_SMB'] = np.nan
        resultados['Beta_SMB (p-valor)'] = np.nan
        
    if 'HML' in params:
        resultados['Beta_HML'] = params['HML']
        resultados['Beta_HML (p-valor)'] = pvalues['HML']
    else:
        resultados['Beta_HML'] = np.nan
        resultados['Beta_HML (p-valor)'] = np.nan
    
    return resultados

In [60]:
resultados_lista = [
    extrair_resultados(modelo_capm_varejo, 'CAPM Varejo'),
    extrair_resultados(modelo_ff_varejo, 'Fama-French Varejo'),
    extrair_resultados(modelo_capm_bancos, 'CAPM Bancos'),
    extrair_resultados(modelo_ff_bancos, 'Fama-French Bancos'),
    extrair_resultados(modelo_capm_eletrico, 'CAPM Elétrico'),
    extrair_resultados(modelo_ff_eletrico, 'Fama-French Elétrico')
]

df_resultados = pd.DataFrame(resultados_lista)
pd.options.display.float_format = '{:.4f}'.format


## Análise Comparativa

In [61]:
# Comparação de R² 

r2_data = pd.DataFrame({
    'CAPM': [modelo_capm_varejo.rsquared, modelo_capm_bancos.rsquared, modelo_capm_eletrico.rsquared],
    'Fama-French': [modelo_ff_varejo.rsquared, modelo_ff_bancos.rsquared, modelo_ff_eletrico.rsquared]
}, index=['Varejo', 'Bancos', 'Elétrico'])

In [62]:
print("Ganho com Fama-French:")

ganho_varejo = modelo_ff_varejo.rsquared - modelo_capm_varejo.rsquared
pct_varejo = (ganho_varejo / modelo_capm_varejo.rsquared) * 100

print(f"\nVarejo:")
print(f"  CAPM R²: {modelo_capm_varejo.rsquared:.4f}")
print(f"  FF R²: {modelo_ff_varejo.rsquared:.4f}")
print(f"  Ganho: {ganho_varejo:.4f} ({pct_varejo:.2f}%)")

ganho_bancos = modelo_ff_bancos.rsquared - modelo_capm_bancos.rsquared
pct_bancos = (ganho_bancos / modelo_capm_bancos.rsquared) * 100

print(f"\nBancos:")
print(f"  CAPM R²: {modelo_capm_bancos.rsquared:.4f}")
print(f"  FF R²: {modelo_ff_bancos.rsquared:.4f}")
print(f"  Ganho: {ganho_bancos:.4f} ({pct_bancos:.2f}%)")

ganho_eletrico = modelo_ff_eletrico.rsquared - modelo_capm_eletrico.rsquared
pct_eletrico = (ganho_eletrico / modelo_capm_eletrico.rsquared) * 100

print(f"\nElétrico:")
print(f"  CAPM R²: {modelo_capm_eletrico.rsquared:.4f}")
print(f"  FF R²: {modelo_ff_eletrico.rsquared:.4f}")
print(f"  Ganho: {ganho_eletrico:.4f} ({pct_eletrico:.2f}%)")

Ganho com Fama-French:

Varejo:
  CAPM R²: 0.3003
  FF R²: 0.3683
  Ganho: 0.0680 (22.65%)

Bancos:
  CAPM R²: 0.5183
  FF R²: 0.5468
  Ganho: 0.0286 (5.51%)

Elétrico:
  CAPM R²: 0.4106
  FF R²: 0.4206
  Ganho: 0.0099 (2.42%)


## Comparação de Betas

In [63]:
beta_data = pd.DataFrame({
    'Beta (CAPM)': [
        modelo_capm_varejo.params['Rm_minus_Rf'],
        modelo_capm_bancos.params['Rm_minus_Rf'],
        modelo_capm_eletrico.params['Rm_minus_Rf']
    ],
    'Beta (Fama-French)': [
        modelo_ff_varejo.params['Rm_minus_Rf'],
        modelo_ff_bancos.params['Rm_minus_Rf'],
        modelo_ff_eletrico.params['Rm_minus_Rf']
    ]
}, index=['Varejo', 'Bancos', 'Elétrico'])

In [64]:
print('Sensibilidade ao Mercado (Beta):\n')
display(beta_data)

Sensibilidade ao Mercado (Beta):



,Beta (CAPM),Beta (Fama-French)
Varejo,1.4347,1.2813
Bancos,0.9085,0.9242
Elétrico,0.7065,0.7035


## Análise dos Alphas

In [65]:
alpha_data = pd.DataFrame({
    'Alpha (CAPM)': [
        modelo_capm_varejo.params['const'],
        modelo_capm_bancos.params['const'],
        modelo_capm_eletrico.params['const']
    ],
    'p-valor (CAPM)': [
        modelo_capm_varejo.pvalues['const'],
        modelo_capm_bancos.pvalues['const'],
        modelo_capm_eletrico.pvalues['const']
    ],
    'Alpha (FF)': [
        modelo_ff_varejo.params['const'],
        modelo_ff_bancos.params['const'],
        modelo_ff_eletrico.params['const']
    ],
    'p-valor (FF)': [
        modelo_ff_varejo.pvalues['const'],
        modelo_ff_bancos.pvalues['const'],
        modelo_ff_eletrico.pvalues['const']
    ]
}, index=['Varejo', 'Bancos', 'Elétrico'])

## Análise de Fatores SMB e HML

In [66]:
fatores_ff = pd.DataFrame({
    'Beta_SMB': [
        modelo_ff_varejo.params['SMB'],
        modelo_ff_bancos.params['SMB'],
        modelo_ff_eletrico.params['SMB']
    ],
    'p-valor SMB': [
        modelo_ff_varejo.pvalues['SMB'],
        modelo_ff_bancos.pvalues['SMB'],
        modelo_ff_eletrico.pvalues['SMB']
    ],
    'Beta_HML': [
        modelo_ff_varejo.params['HML'],
        modelo_ff_bancos.params['HML'],
        modelo_ff_eletrico.params['HML']
    ],
    'p-valor HML': [
        modelo_ff_varejo.pvalues['HML'],
        modelo_ff_bancos.pvalues['HML'],
        modelo_ff_eletrico.pvalues['HML']
    ]
}, index=['Varejo', 'Bancos', 'Elétrico'])

print("FATORES TAMANHO (SMB) E VALOR (HML):\n")
display(fatores_ff)


FATORES TAMANHO (SMB) E VALOR (HML):



,Beta_SMB,p-valor SMB,Beta_HML,p-valor HML
Varejo,0.7190,0.0000,-0.4799,0.0000
Bancos,-0.0935,0.0003,0.3771,0.0000
Elétrico,0.0012,0.9613,0.1972,0.0000


## Conclusões e gráficos

In [67]:
print("Poder Explicativo dos Modelos:")

if modelo_ff_varejo.rsquared > modelo_capm_varejo.rsquared:
    print(' Fama-French explicar MELHOR o setor Varejista.')
else:
    print(' CAPM explicar MELHOR o setor Varejista.')

if modelo_ff_bancos.rsquared > modelo_capm_bancos.rsquared:
    print("✓ Fama-French explica MELHOR o setor BANCÁRIO")
else:
    print("✓ CAPM explica MELHOR o setor BANCÁRIO")

if modelo_ff_eletrico.rsquared > modelo_capm_eletrico.rsquared:
    print("✓ Fama-French explica MELHOR o setor ELÉTRICO")
else:
    print("✓ CAPM explica MELHOR o setor ELÉTRICO")

Poder Explicativo dos Modelos:
 Fama-French explicar MELHOR o setor Varejista.
✓ Fama-French explica MELHOR o setor BANCÁRIO
✓ Fama-French explica MELHOR o setor ELÉTRICO


In [68]:
print(f"\nDetalhes:")
print(f"Varejo - CAPM R²: {modelo_capm_varejo.rsquared:.3f}, Fama-French R²: {modelo_ff_varejo.rsquared:.3f}")
print(f"Bancos - CAPM R²: {modelo_capm_bancos.rsquared:.3f}, Fama-French R²: {modelo_ff_bancos.rsquared:.3f}")
print(f"Elétrico - CAPM R²: {modelo_capm_eletrico.rsquared:.3f}, Fama-French R²: {modelo_ff_eletrico.rsquared:.3f}")


Detalhes:
Varejo - CAPM R²: 0.300, Fama-French R²: 0.368
Bancos - CAPM R²: 0.518, Fama-French R²: 0.547
Elétrico - CAPM R²: 0.411, Fama-French R²: 0.421


In [69]:
r2_data = pd.DataFrame({
    'Setor': ['Varejo', 'Varejo', 'Bancos', 'Bancos', 'Elétrico', 'Elétrico'],
    'Modelo': ['CAPM', 'Fama-French', 'CAPM', 'Fama-French', 'CAPM', 'Fama-French'],
    'R2': [modelo_capm_varejo.rsquared, modelo_ff_varejo.rsquared, 
           modelo_capm_bancos.rsquared, modelo_ff_bancos.rsquared,
           modelo_capm_eletrico.rsquared, modelo_ff_eletrico.rsquared]
})

fig3 = px.bar(r2_data, x='Setor', y='R2', color='Modelo', barmode='group',
             title='Comparação do Poder Explicativo (R²)')
fig3.show()

# Evolução das Carteira

In [70]:
retornos_acumulados = pd.DataFrame({
    'Varejo': (1 + carteira_varejo).cumprod(),
    'Bancos': (1 + carteira_bancos).cumprod(),
    'Elétrico': (1 + carteira_eletrico).cumprod(),
    'Mercado': (1 + retorno_mercado).cumprod()
})

In [71]:
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=retornos_acumulados.index, y=retornos_acumulados['Varejo'],
                         mode='lines', name='Carteira Varejo', line=dict(color='red')))
fig1.add_trace(go.Scatter(x=retornos_acumulados.index, y=retornos_acumulados['Bancos'],
                         mode='lines', name='Carteira Bancos', line=dict(color='blue')))
fig1.add_trace(go.Scatter(x=retornos_acumulados.index, y=retornos_acumulados['Mercado'],
                         mode='lines', name='Mercado', line=dict(color='gray', dash='dash')))
fig1.add_trace(go.Scatter(x=retornos_acumulados.index, y=retornos_acumulados['Elétrico'],
                         mode='lines', name='Carteira Elétrico', line=dict(color='green')))

fig1.update_layout(title='Evolução das Carteiras Setoriais',
                  xaxis_title='Data', yaxis_title='Retorno Acumulado')
fig1.show()

# Retorno dos Tickers Individuais

In [72]:
retornos_acum_tickers = pd.DataFrame({
    'MGLU3': (1 + retornos_varejo['MGLU3.SA']).cumprod(),
    'LREN3': (1 + retornos_varejo['LREN3.SA']).cumprod(),
    'BHIA3': (1 + retornos_varejo['BHIA3.SA']).cumprod(),
    'ITSA4': (1 + retornos_bancos['ITSA4.SA']).cumprod(),
    'BBAS3': (1 + retornos_bancos['BBAS3.SA']).cumprod(),
    'BBDC4': (1 + retornos_bancos['BBDC4.SA']).cumprod(),
    'ELET3': (1 + retornos_eletrico['ELET3.SA']).cumprod(),
    'TAEE11': (1 + retornos_eletrico['TAEE11.SA']).cumprod(),
    'CMIG4': (1 + retornos_eletrico['CMIG4.SA']).cumprod()
})

In [73]:
fig2 = make_subplots(rows=3, cols=1, subplot_titles=('Setor Varejo', 'Setor Bancário', 'Setor Elétrico'))

# Varejo
fig2.add_trace(go.Scatter(x=retornos_acum_tickers.index, y=retornos_acum_tickers['MGLU3'],
                         mode='lines', name='MGLU3'), row=1, col=1)
fig2.add_trace(go.Scatter(x=retornos_acum_tickers.index, y=retornos_acum_tickers['LREN3'],
                         mode='lines', name='LREN3'), row=1, col=1)
fig2.add_trace(go.Scatter(x=retornos_acum_tickers.index, y=retornos_acum_tickers['BHIA3'],
                         mode='lines', name='BHIA3'), row=1, col=1)

# Bancos
fig2.add_trace(go.Scatter(x=retornos_acum_tickers.index, y=retornos_acum_tickers['ITSA4'],
                         mode='lines', name='ITSA4'), row=2, col=1)
fig2.add_trace(go.Scatter(x=retornos_acum_tickers.index, y=retornos_acum_tickers['BBAS3'],
                         mode='lines', name='BBAS3'), row=2, col=1)
fig2.add_trace(go.Scatter(x=retornos_acum_tickers.index, y=retornos_acum_tickers['BBDC4'],
                         mode='lines', name='BBDC4'), row=2, col=1)

# Elétrico
fig2.add_trace(go.Scatter(x=retornos_acum_tickers.index, y=retornos_acum_tickers['ELET3'],
                         mode='lines', name='ELET3'), row=3, col=1)
fig2.add_trace(go.Scatter(x=retornos_acum_tickers.index, y=retornos_acum_tickers['TAEE11'],
                         mode='lines', name='TAEE11'), row=3, col=1)
fig2.add_trace(go.Scatter(x=retornos_acum_tickers.index, y=retornos_acum_tickers['CMIG4'],
                         mode='lines', name='CMIG4'), row=3, col=1)

fig2.update_layout(height=900, title_text="Retornos Acumulados por Ticker")
fig2.show()


# Sensibilidade de Mercado

In [74]:
betas_data = pd.DataFrame({
    'Setor': ['Varejo', 'Bancos', 'Elétrico'],
    'Beta CAPM': [modelo_capm_varejo.params['Rm_minus_Rf'], 
                  modelo_capm_bancos.params['Rm_minus_Rf'],
                  modelo_capm_eletrico.params['Rm_minus_Rf']],
    'Beta FF': [modelo_ff_varejo.params['Rm_minus_Rf'], 
                modelo_ff_bancos.params['Rm_minus_Rf'],
                modelo_ff_eletrico.params['Rm_minus_Rf']]
})

fig4 = px.bar(betas_data, x='Setor', y=['Beta CAPM', 'Beta FF'],
             title='Sensibilidade ao Mercado (Betas)', barmode='group')
fig4.show()

# Retornos Anormais

In [75]:
alphas_data = pd.DataFrame({
    'Setor': ['Varejo', 'Varejo', 'Bancos', 'Bancos', 'Elétrico', 'Elétrico'],
    'Modelo': ['CAPM', 'Fama-French', 'CAPM', 'Fama-French', 'CAPM', 'Fama-French'],
    'Alpha': [modelo_capm_varejo.params['const'], modelo_ff_varejo.params['const'],
              modelo_capm_bancos.params['const'], modelo_ff_bancos.params['const'],
              modelo_capm_eletrico.params['const'], modelo_ff_eletrico.params['const']]
})

fig5 = px.bar(alphas_data, x='Setor', y='Alpha', color='Modelo', barmode='group',
             title='Retornos Anormais (Alphas)')
fig5.add_hline(y=0, line_dash="dash", line_color="red")
fig5.show()

# Resumo Final

In [76]:
melhor_varejo = "Fama-French" if modelo_ff_varejo.rsquared > modelo_capm_varejo.rsquared else "CAPM"
melhor_bancos = "Fama-French" if modelo_ff_bancos.rsquared > modelo_capm_bancos.rsquared else "CAPM"
melhor_eletrico = "Fama-French" if modelo_ff_eletrico.rsquared > modelo_capm_eletrico.rsquared else "CAPM"

print(f"Modelo que melhor explica Varejo : {melhor_varejo}")
print(f"Modelo que melhor explica Bancos : {melhor_bancos}")
print(f"Modelo que melhor explica Elétrico: {melhor_eletrico}")


volatilidades = {
    'Varejo': carteira_varejo.std(),
    'Bancos': carteira_bancos.std(),
    'Elétrico': carteira_eletrico.std()
}
setor_mais_volatil = max(volatilidades, key=volatilidades.get)

print("\nSetores - Volatilidade (Desvio Padrão):")
for setor, vol in volatilidades.items():
    print(f"  {setor}: {vol:.6f}")
print(f"=> Setor mais volátil: {setor_mais_volatil}")

retornos_finais = retornos_acumulados.iloc[-1][['Varejo', 'Bancos', 'Elétrico']]
setor_melhor_desempenho = retornos_finais.idxmax()

print("\nRetornos Acumulados Finais:")
for setor, ret in retornos_finais.items():
    print(f"  {setor}: {ret:.6f}")
print(f"=> Melhor desempenho: {setor_melhor_desempenho}")




Modelo que melhor explica Varejo : Fama-French
Modelo que melhor explica Bancos : Fama-French
Modelo que melhor explica Elétrico: Fama-French

Setores - Volatilidade (Desvio Padrão):
  Varejo: 0.032344
  Bancos: 0.015590
  Elétrico: 0.013615
=> Setor mais volátil: Varejo

Retornos Acumulados Finais:
  Varejo: 0.109136
  Bancos: 1.903814
  Elétrico: 2.471543
=> Melhor desempenho: Elétrico
